Calculator

- encoder로 추출한 output 파일과 input 값의 embedding 값들의 코사인 유사도를 계산



In [1]:
pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 마운트
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# 데이터를 NumPy array로 변환하는 함수
def parse_and_convert(value):

    value = value.strip('[]')
    return np.array([float(x) for x in value.split(',')])

In [12]:
# output 파일 로드
csv_file_path = 'output_first.csv'  # Replace with the actual path to your CSV file
# csv_file_path = 'output_total.csv'  # Replace with the actual path to your CSV file
df = pd.read_csv(csv_file_path)

# NumPy array로 변환
df['embeddings'] = df['embeddings'].apply(parse_and_convert)

# 타겟값 지정
target_column = 'embeddings'
target_data = df[target_column]

In [4]:
df.iloc[[1,3,5,7,9]]
da = df.drop([1,3,5,7,9])

In [ ]:
df.shape

(40128, 4)

CASE1. 초록을 새로 입력 시 가장 유사한 초록과 그 ipc코드를 반환


In [ ]:
df['초록'][100]

'본 발명은 창틀과 윈도우 사이에 신체의 일부나 물체가 끼일 경우 윈도우를 강제로 다운시킬 수 있는 파워 윈도우의 안전장치 및 제어방법에 관한 것이다.본 발명에 따른 파워 윈도우의 안전장치는 자동차의 창틀의 고무패킹 내부에 장착되어 상기 창틀과 윈도우 사이에 물체가 끼일 경우 상기 윈도우를 하강시키도록 하는 파워 윈도우 안전장치에 있어서  창틀의 내측 일정위치에 형성되어 윈도우를 업 시키거나 또는 다운시키기 위한 스위치모듈과  창틀 상단부 내부에 다수 설치되어 스위치모듈의 작동으로 업 되는 윈도우와 창틀과 사이에 물체가 끼면 ECU로 신호를 전송하는 안전스위치부와  안전스위치부로부터 인가되는 신호에 의해 윈도우를 다운시키도록 구동되는 윈도우모터와  도어의 하부 내측에 형성되어 상기 윈도우를 일정거리 다운되면 윈도우모터의 구동을 정지시키기 위한 오토다운센서와  스위치모듈과 안전스위치부 및 오토다운센서로부터 인가되는 신호에 따라 윈도우모터의 구동을 제어하는 제어ECU를 포함하여 구성되어 그 안전장치를 제어하는 제어방법을 특징으로 한다.    도어  윈도우  안전스위치'

In [89]:
# 평균 풀링 - 주의 마스크(attention_mask)를 고려하여 평균 계산
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# 학습된 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('jhgan/ko-sroberta-multitask')
model = AutoModel.from_pretrained('jhgan/ko-sroberta-multitask')

seed = 10042    
a = df.초록[seed]
df = df.drop(seed)
# 입력 문장
input_sentence = a

# 입력문장 인코딩
encoded_input = tokenizer(input_sentence, padding=True, truncation=True, return_tensors='pt')
with torch.no_grad():
    model_output = model(**encoded_input)
input_embedding = mean_pooling(model_output, encoded_input['attention_mask']).numpy()


# 코사인 유사도 계산
similarity_scores = cosine_similarity(input_embedding, np.stack(target_data.apply(np.array), axis=0))


# 가장 유사한 인덱스 추출
most_similar_row_index = np.argmax(similarity_scores)
result_1 = df.loc[most_similar_row_index, '초록']
result_2 = df.loc[most_similar_row_index, 'ipc코드']

print(a)
# 결과 도출
print("\n가장 유사한 초록 \n->",result_1)
print("\n유사 ipc코드\n->",result_2)

본 발명은 냉장고용 탈취 촉매필터에 관한 것으로  좀 더 상세하게는 다공질의 무기질 섬유로 이루어져 80 130  g의 비표면적을 갖는 주름진 형태의 담체와  그리고 150 300  g의 비표면적을 갖는 MnO2와 150 300  g의 비표면적을 갖는 CuO의 혼합물로 이루어져 상기 담체에 담지되어 있는 촉매로 이루어지므로써 종래의 것에 비해 탈취 성능이 현저히 뛰어나며 수명이 반영구적인 잇점이 있다. 

가장 유사한 초록 
-> 본 발명은 원료육을 신속하게 염지하는 방법에 관한 것으로서  쇠고기  돼지고기  닭고기등 육류덩어리고기 직경 100mm 120mm 에 간장  설탕  마늘즙  생강즙등 양념을 신속하게 침투시키기 위하여 가압텀블링을 이용하여 원료육을 신속하게 염지함으로써 원료육을 손상치 않고 보다 신속하고 효과적으로 염지처리하여 큰 덩어리육의 염지처리 공정에 다양하게 이용할 수 있어 육가공 관련 산업의 발전에 크게 기여할 수 있는 패스트푸드용 불고기 원료육의 처리방법이다.

유사 ipc코드
-> A23L


In [15]:
df['초록'][0].startswith('본 발명은 ')

True

In [21]:
import pandas as pd
df = pd.read_csv('../data/Final_data.csv')
# df는 당신이 가지고 있는 DataFrame 객체입니다.

for index, row in df.iterrows():
    if '식물가해곤충 ' in row['초록']:
        print(row['ipc코드'])


A01N
C07F


In [24]:
df[df['ipc코드'] == 'A01N']

,출원번호,초록,ipc코드
0,1019790001108,quot 내용 없음 quot,A01N
1,1019820000038,목적 구조식 I 의 N 할로알킬 티오벤즈아닐리드 살균제 및 그 제조방법을 제공하는...,A01N
10,1019840000112,목적 광범위한 공업재료를 보존하기에 적합하고 안존한 보존제인 향균 작용을 갖는 o...,A01N
14,1019840003269,목적 신규의 1 옥소 이소인돌린 유도체의 제법을 제공함을 목적으로 한다. 구성 일...,A01N
17,1019840004705,목적 본 발명은 살충제로 유용한 신규한 물질인 카바메이트 화합물 의 제조방법을 제...,A01N
...,...,...,...
23050,1019970708411,식 I 의 화합물 및 이로부터 유도된 산 부가염 및 4차 암모늄 염 및 N ...,A01N
23052,1019970708417,본 발명은 반시류 곤충 해충을 박멸하기 위해 바실루스 츄린겐시스 비.티. 내독소...,A01N
23061,1019970708543,본 발명은 알콜 함량이 낮으며 표면의 신속한 1단계 소독 및 건조가 가능한 박테리...,A01N
23062,1019970708584,본 발명은 하기 일반식 1 으로 나타낸 페닐알칸아미드 유도체 및 이를 유효성분으로서...,A01N


CASE2. ipc 코드 맞출 확률 계산

In [6]:
from tqdm import tqdm
# 확률계산 함수 설정
def calculate_matching_ipc_probability(df, tokenizer, model, num):
    matching_count = 0
    big_count = 0
    total_count = 0
    
    A = 0
    B = 0
    C = 0
    D = 0
    E = 0
    F = 0
    G = 0
    
    a = 0
    b = 0
    c = 0
    d = 0
    e = 0
    f = 0
    g = 0
    
    a_same = 0
    b_same = 0
    c_same = 0
    d_same = 0
    e_same = 0
    f_same = 0
    g_same = 0
    
    
    
    for _ in tqdm(range(num)):
        # df 내 100개 데이터 랜덤 추출

        random_index = np.random.randint(len(df)-1)
        random_row = df.iloc[random_index]
        df = df.drop(random_index)
        df = df.reset_index(drop = True)
        
        # 입력 문장
        input_sentence = random_row['초록']

        # 입력 문장 인코딩
        encoded_input = tokenizer(input_sentence, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = model(**encoded_input)
        input_embedding = mean_pooling(model_output, encoded_input['attention_mask']).numpy()

        # 코사인 유사도 계산
        similarity_scores = cosine_similarity(input_embedding, np.stack(df['embeddings'].to_numpy(), axis=0))

        # 가장 유사한 인덱스 추출
        most_similar_row_index = np.argmax(similarity_scores)
        most_similar_row = df.iloc[most_similar_row_index]

        # ipc코드 매칭 여부
        ran_IPC = random_row['ipc코드']
        sim_IPC = most_similar_row['ipc코드']
        
        if ran_IPC == sim_IPC:
            matching_count += 1
        if ran_IPC[0] == sim_IPC[0]:
            big_count += 1
        
        if ran_IPC[0] == 'A':
            A += 1
            if sim_IPC[0] == 'A':
                a += 1
                if ran_IPC == sim_IPC:
                    a_same += 1
        if ran_IPC[0] == 'B':
            B += 1
            if sim_IPC[0] == 'B':
                b += 1
                if ran_IPC == sim_IPC:
                    b_same += 1                
        if ran_IPC[0] == 'C':
            C += 1
            if sim_IPC[0] == 'C':
                c += 1
                if ran_IPC == sim_IPC:
                    c_same += 1                
        if ran_IPC[0] == 'D':
            D += 1
            if sim_IPC[0] == 'D':
                d += 1
                if ran_IPC == sim_IPC:
                    d_same += 1                
        if ran_IPC[0] == 'E':
            E += 1
            if sim_IPC[0] == 'E':
                e += 1
                if ran_IPC == sim_IPC:
                    e_same += 1                
        if ran_IPC[0] == 'F':
            F += 1
            if sim_IPC[0] == 'F':
                f += 1
                if ran_IPC == sim_IPC:
                    f_same += 1                
        if ran_IPC[0] == 'G':
            G += 1
            if sim_IPC[0] == 'G':
                g += 1
                if ran_IPC == sim_IPC:
                    g_same += 1                
                                                                                                   
        

        total_count += 1
        # 복원
        df = df.append(random_row)
    
    big_probability = big_count / total_count
    matching_probability = matching_count / total_count
    last = [a/A, b/B, c/C, d/D, e/E, f/F, g/G]
    semi = [a_same/a, b_same/b, c_same/c, d_same/d, e_same/e, f_same/f, g_same/g]
    print('*' * 30)
    print(f"ipc코드 4자리 맞출 확률 : {matching_probability*100}'%' ")
    print(f"ipc코드 대분류 맞출 확률 : {big_probability*100}'%' ")
    print(f"소분류 A 맞출 확률 : {last[0]*100}'%' ")
    print(f"소분류 B 맞출 확률 : {last[1]*100}'%' ")
    print(f"소분류 C 맞출 확률 : {last[2]*100}'%' ")
    print(f"소분류 D 맞출 확률 : {last[3]*100}'%' ")
    print(f"소분류 E 맞출 확률 : {last[4]*100}'%' ")
    print(f"소분류 F 맞출 확률 : {last[5]*100}'%' ")
    print(f"소분류 F 맞출 확률 : {last[6]*100}'%' ")
    print('*' * 30)
    print(f"4자리 맞출 확률 : {(a_same + b_same + c_same + d_same + e_same + f_same + g_same)/(a+b+c+d+e+f+g)}")
    print(f"A 4자리 맞출 확률 : {semi[0]*100}'%' ")
    print(f"B 4자리 맞출 확률 : {semi[1]*100}'%' ")
    print(f"C 4자리 맞출 확률 : {semi[2]*100}'%' ")
    print(f"D 4자리 맞출 확률 : {semi[3]*100}'%' ")
    print(f"E 4자리 맞출 확률 : {semi[4]*100}'%' ")
    print(f"F 4자리 맞출 확률 : {semi[5]*100}'%' ")
    print(f"F 4자리 맞출 확률 : {semi[6]*100}'%' ")    
    
    return matching_probability, big_probability, last




In [61]:
import random
import warnings
warnings.filterwarnings('ignore')

seed = 0

random.seed(seed)
np.random.seed(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(seed)
if device == 'cuda':
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# 결과 도출
matching_probability, big_probability, last = calculate_matching_ipc_probability(df.copy(), tokenizer, model, 1000)

100%|██████████| 1000/1000 [06:36<00:00,  2.52it/s]

******************************
ipc코드 4자리 맞출 확률 : 30.599999999999998'%' 
ipc코드 대분류 맞출 확률 : 62.3'%' 
소분류 A 맞출 확률 : 58.857142857142854'%' 
소분류 B 맞출 확률 : 56.61764705882353'%' 
소분류 C 맞출 확률 : 75.21367521367522'%' 
소분류 D 맞출 확률 : 55.55555555555556'%' 
소분류 E 맞출 확률 : 62.7906976744186'%' 
소분류 F 맞출 확률 : 60.71428571428571'%' 
소분류 F 맞출 확률 : 68.39080459770115'%' 
******************************
4자리 맞출 확률 : 0.4911717495987159
A 4자리 맞출 확률 : 66.99029126213593'%' 
B 4자리 맞출 확률 : 37.66233766233766'%' 
C 4자리 맞출 확률 : 43.18181818181818'%' 
D 4자리 맞출 확률 : 55.00000000000001'%' 
E 4자리 맞출 확률 : 55.55555555555556'%' 
F 4자리 맞출 확률 : 50.588235294117645'%' 
F 4자리 맞출 확률 : 47.89915966386555'%' 
